#torch.save(my_net.state_dict(), '/cnn_state')
#my_net.load_state_dict(torch.load('/cnn_state'))

In [1]:
from torchvision.models import resnet34

In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import load_run_nets

In [4]:
import torch
import torch as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

In [5]:
net = load_run_nets.load_resnet(data_list=[], num_net=34, classifier=True, csv_file=False, num_labels=76)

In [6]:
df = pd.read_csv('dataset_sample.csv')
df = df.dropna()

In [7]:
enc = LabelEncoder()
labels = enc.fit_transform(df['subcategory_id'])

In [8]:
tick = {x:y for x, y in zip(labels, df.subcategory_id)}
tick1 = { str(tick[x]) : x for x in range(76)}
label = pd.DataFrame(tick1, index=[0])

In [9]:
mas = np.array(df.image_links)

In [10]:
X, Y = mas, labels
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [11]:
def data2image(data):
    return Image.fromarray(data)

In [12]:
def dataloader(urls):
    array = []
    width = 224
    height = 224
    
    for url in urls:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((width, height), Image.ANTIALIAS)
        img = np.array(img).transpose()
        array.append(img)
    data = nn.FloatTensor(array)
    return data

In [13]:
def prediction2classes(output_var):
    _, predicted = nn.max(output_var.data, 1)
    predicted.squeeze_()
    classes = predicted.tolist()
    return classes

In [14]:
def train_network(a_net, 
                  a_device,
                  a_epochs=100,
                  a_batch_size=32,
                  a_lr=0.1):
    
    train_acc = []
    net = a_net.to(a_device)

    criterion = nn.nn.CrossEntropyLoss()
    optimizer = nn.optim.SGD(a_net.parameters(), lr=a_lr, weight_decay=0.0001, momentum=0.9)

    
    for epoch in range(a_epochs):  # loop over the dataset multiple times
        if epoch == 42:
            optimizer = nn.optim.SGD(a_net.parameters(), lr=a_lr/10, weight_decay=0.0001, momentum=0.9) 
        elif epoch == 83:
            optimizer = nn.optim.SGD(a_net.parameters(), lr=a_lr/100, weight_decay=0.0001, momentum=0.9) 
        
        net = net.train().to(a_device)        
        epoch_accuracy = 0.0
        epoch_iters = 0
        
        for it in range(X_train.shape[0]//a_batch_size + 1):
            
            epoch_iters += 1
            inputs = dataloader(X_train[it*a_batch_size:it*a_batch_size+a_batch_size]).to(a_device)/255
            labels = nn.FloatTensor(y_train[it*a_batch_size:it*a_batch_size+a_batch_size]).long().to(a_device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

#             epoch_accuracy += accuracy_score(labels, prediction2classes(outputs))

#         epoch_accuracy /= epoch_iters
#         train_acc.append(epoch_accuracy)
        
        print("Epoch ", epoch, " Loss: ", loss)

    print('Finished Training')
    
    plt.plot(loss, label='Train')
    plt.legend()
    plt.grid()

In [52]:
DEVICE_ID = 0
DEVICE = torch.device('cuda:%d' % DEVICE_ID)
torch.cuda.set_device(DEVICE_ID)
DEVICE_ID

0

In [53]:
import os
print(os.getpid())

12960


In [60]:
%%time
train_network(a_net=net, a_device=DEVICE_ID, a_epochs=1, a_batch_size=64)


Epoch  0  Loss:  tensor(6.7408, device='cuda:0', grad_fn=<NllLossBackward>)
Finished Training


NameError: name 'plt' is not defined

In [61]:
torch.save(net.state_dict(), './cnn_state_1')